In [141]:
import pandas as pd
import numpy as np
import pickle
import os
import json
from IPython.display import display

In [150]:
[i for i, v in enumerate(valid_acc) if top == v][0]

6

In [161]:
nfiles = 4
data = []
for base_fn in ['out_{}_4', 'out_{}_4_false_false',  'out_{}_4_false_true',  'out_{}_4_true_true']:
    for file_idx in range(nfiles):
        with open((base_fn + '.log').format(file_idx, nfiles), 'r') as rf:
            lines = rf.readlines()
        results = "".join(lines).split('-------  ------------  -----------  ------------  ------')[1:]
        for r in results:
            epoch = r.split()[0::5][:30]
            train_loss = r.split()[1::5][:30]
            valid_acc = r.split()[2::5][:30]
            valid_loss = r.split()[3::5][:30]
            top = [z for z in valid_acc if 'm' in z][-1].split('2m')[-1].split('\x1b[0m')[0]
            final = valid_acc[-1].split('2m')[-1].split('\x1b[0m')[0]
            valid_loss = [z.split('5m')[-1].split('\x1b[0m')[0] for z in valid_loss]
            train_loss = [z.split('6m')[-1].split('\x1b[0m')[0] for z in train_loss]
            valid_acc = [z.split('2m')[-1].split('\x1b[0m')[0] for z in valid_acc]
            setting = {'dataset': base_fn}
            setting.update(json.loads(r.split('settings')[1].split('\n')[0].replace("'",'"')))
            setting['top_score'] = float(top)
            setting['epoch_of_top_score'] = [i for i, v in enumerate(valid_acc) if top == v][0]
            setting['final_score'] = float(final)
            setting['combined_score'] = (float(top) + float(final)) / 2
            setting['train_loss'] = train_loss
            setting['valid_acc'] = valid_acc
            setting['valid_loss'] = valid_loss
            data.append(setting)

In [162]:
df = pd.DataFrame(data)

In [164]:
df.groupby(
    ['wd', 'initial_lr', 'eta_min', 'num_hidden_layers', 'dropout', 'momentum']
).sum().sort_values('top_score', ascending=False).head(20)

top_score  \
wd     initial_lr eta_min  num_hidden_layers dropout momentum              
0.0100 0.10000    0.000001 1                 0.5     0.9          2.0458   
                  0.001000 1                 0.5     0.9          2.0430   
                  0.000100 1                 0.5     0.9          2.0421   
                  0.000010 1                 0.5     0.9          2.0392   
                                             0.2     0.9          2.0384   
                  0.000100 1                 0.1     0.9          2.0361   
                  0.001000 1                 0.1     0.9          2.0345   
                                             0.2     0.9          2.0343   
                  0.000001 1                 0.1     0.9          2.0328   
0.0001 0.00001    0.001000 2                 0.5     0.9          2.0325   
       0.00010    0.000010 2                 0.5     0.9          2.0325   
0.0100 0.10000    0.000001 1                 0.2     0.9          2.0319   
0.0001 0.10000    0.000100 1                 0.5     0.9          2.0315   
0.0010 0.00010    0.000001 1                 0.5     0.9          2.0311   
                  0.000100 2                 0.5     0.9          2.0307   
0.0100 0.10000    0.000010 1                 0.1     0.9          2.0306   
       0.00001    0.001000 2                 0.5     0.9          2.0300   
0.0010 0.00100    0.000001 2                 0.5     0.9          2.0296   
       0.00010    0.001000 2                 0.5     0.9          2.0287   
0.0100 0.00010    0.000100 2                 0.5     0.9          2.0286   

                                                               epoch_of_top_score  \
wd     initial_lr eta_min  num_hidden_layers dropout momentum                       
0.0100 0.10000    0.000001 1                 0.5     0.9                       80   
                  0.001000 1                 0.5     0.9                       80   
                  0.000100 1                 0.5     0.9                       79   
                  0.000010 1                 0.5     0.9                       82   
                                             0.2     0.9                       81   
                  0.000100 1                 0.1     0.9                       80   
                  0.001000 1                 0.1     0.9                       81   
                                             0.2     0.9                       82   
                  0.000001 1                 0.1     0.9                       83   
0.0001 0.00001    0.001000 2                 0.5     0.9                       43   
       0.00010    0.000010 2                 0.5     0.9                       43   
0.0100 0.10000    0.000001 1                 0.2     0.9                       81   
0.0001 0.10000    0.000100 1                 0.5     0.9                       59   
0.0010 0.00010    0.000001 1                 0.5     0.9                       45   
                  0.000100 2                 0.5     0.9                       63   
0.0100 0.10000    0.000010 1                 0.1     0.9                       84   
       0.00001    0.001000 2                 0.5     0.9                       40   
0.0010 0.00100    0.000001 2                 0.5     0.9                        9   
       0.00010    0.001000 2                 0.5     0.9                       23   
0.0100 0.00010    0.000100 2                 0.5     0.9                       49   

                                                               final_score  \
wd     initial_lr eta_min  num_hidden_layers dropout momentum                
0.0100 0.10000    0.000001 1                 0.5     0.9            2.0291   
                  0.001000 1                 0.5     0.9            2.0277   
                  0.000100 1                 0.5     0.9            2.0248   
                  0.000010 1                 0.5     0.9            2.0282   
                                             0.2   

In [155]:
for i, sdf in df.groupby('dataset'):
    print(i)
    display(sdf.sort_values('top_score', ascending=False)[:15])
    for ft in ['wd', 'initial_lr', 'eta_min', 'num_hidden_layers', 'dropout', 'momentum']:
        print(ft)
        print(sdf.sort_values('top_score', ascending=False)[:40][ft].value_counts())
        print()

out_{}_4


,dataset,wd,initial_lr,eta_min,num_hidden_layers,dropout,momentum,top_score,epoch_of_top_score,final_score,combined_score,train_loss,valid_acc,valid_loss
226,out_{}_4,0.0100,0.0001,0.00001,2,0.2,0.9,0.5513,17,0.5470,0.54915,"[1.1760, 1.0670, 1.0227, 0.9946, 0.9778, 0.965...","[0.4937, 0.5142, 0.5248, 0.5341, 0.5381, 0.538...","[1.1154, 1.0843, 1.0700, 1.0631, 1.0590, 1.056..."
418,out_{}_4,0.0001,0.0001,0.00001,2,0.2,0.9,0.5505,16,0.5462,0.54835,"[1.1758, 1.0663, 1.0216, 0.9933, 0.9764, 0.963...","[0.4951, 0.5144, 0.5260, 0.5333, 0.5374, 0.538...","[1.1151, 1.0838, 1.0696, 1.0627, 1.0587, 1.055..."
610,out_{}_4,0.0005,0.0001,0.00001,2,0.2,0.9,0.5505,16,0.5466,0.54855,"[1.1758, 1.0663, 1.0217, 0.9934, 0.9764, 0.963...","[0.4949, 0.5146, 0.5260, 0.5333, 0.5374, 0.538...","[1.1151, 1.0839, 1.0696, 1.0628, 1.0587, 1.055..."
34,out_{}_4,0.0010,0.0001,0.00001,2,0.2,0.9,0.5505,17,0.5466,0.54855,"[1.1762, 1.0667, 1.0217, 0.9939, 0.9770, 0.963...","[0.4953, 0.5152, 0.5254, 0.5327, 0.5391, 0.539...","[1.1152, 1.0839, 1.0697, 1.0629, 1.0589, 1.055..."
669,out_{}_4,0.0005,0.0005,0.00001,3,0.4,0.8,0.5495,12,0.5433,0.54640,"[1.2069, 1.1143, 1.0688, 1.0337, 1.0106, 0.995...","[0.4613, 0.5036, 0.5242, 0.5200, 0.5281, 0.531...","[1.1407, 1.1061, 1.0831, 1.0740, 1.0705, 1.066..."
477,out_{}_4,0.0001,0.0005,0.00001,3,0.4,0.8,0.5493,12,0.5433,0.54630,"[1.2069, 1.1143, 1.0687, 1.0336, 1.0105, 0.995...","[0.4613, 0.5038, 0.5240, 0.5202, 0.5283, 0.532...","[1.1406, 1.1060, 1.0831, 1.0739, 1.0704, 1.066..."
285,out_{}_4,0.0100,0.0005,0.00001,3,0.4,0.8,0.5493,13,0.5461,0.54770,"[1.2074, 1.1157, 1.0716, 1.0370, 1.0140, 0.999...","[0.4605, 0.5005, 0.5254, 0.5219, 0.5260, 0.531...","[1.1416, 1.1078, 1.0849, 1.0749, 1.0712, 1.067..."
93,out_{}_4,0.0010,0.0005,0.00001,3,0.4,0.8,0.5491,12,0.5437,0.54640,"[1.2062, 1.1142, 1.0700, 1.0343, 1.0106, 0.995...","[0.4607, 0.5030, 0.5248, 0.5215, 0.5275, 0.531...","[1.1406, 1.1063, 1.0834, 1.0739, 1.0704, 1.066..."
583,out_{}_4,0.0005,0.0001,0.00005,2,0.6,0.8,0.5489,25,0.5478,0.54835,"[1.2342, 1.1387, 1.1013, 1.0770, 1.0586, 1.043...","[0.4746, 0.5007, 0.5152, 0.5240, 0.5260, 0.530...","[1.1537, 1.1195, 1.1012, 1.0890, 1.0803, 1.074..."
391,out_{}_4,0.0001,0.0001,0.00005,2,0.6,0.8,0.5488,25,0.5478,0.54830,"[1.2342, 1.1387, 1.1013, 1.0770, 1.0585, 1.043...","[0.4748, 0.5007, 0.5152, 0.5240, 0.5260, 0.530...","[1.1537, 1.1195, 1.1012, 1.0890, 1.0803, 1.074..."


wd
0.0010    10
0.0001    10
0.0005    10
0.0100    10
Name: wd, dtype: int64

initial_lr
0.0001    29
0.0005     9
0.0050     2
Name: initial_lr, dtype: int64

eta_min
0.00001    17
0.00005    12
0.00010    11
Name: eta_min, dtype: int64

num_hidden_layers
2    31
3     9
Name: num_hidden_layers, dtype: int64

dropout
0.1    14
0.4    12
0.6    10
0.2     4
Name: dropout, dtype: int64

momentum
0.9    21
0.8    19
Name: momentum, dtype: int64

out_{}_4_false_false


,dataset,wd,initial_lr,eta_min,num_hidden_layers,dropout,momentum,top_score,epoch_of_top_score,final_score,combined_score,train_loss,valid_acc,valid_loss
1004,out_{}_4_false_false,0.0010,0.10000,0.000001,1,0.5,0.9,0.7260,28,0.7245,0.72525,"[2.2174, 0.6483, 0.6500, 0.6481, 0.6496, 0.652...","[0.6474, 0.6474, 0.6473, 0.6473, 0.6473, 0.647...","[0.6337, 0.6522, 0.6522, 0.6561, 0.6376, 0.648..."
991,out_{}_4_false_false,0.0010,0.10000,0.000100,1,0.2,0.9,0.7224,28,0.7219,0.72215,"[2.0819, 0.6505, 0.6474, 0.6521, 0.6513, 0.650...","[0.6474, 0.6473, 0.6473, 0.6473, 0.6473, 0.647...","[0.6581, 0.6460, 0.6480, 0.6371, 0.6256, 0.643..."
998,out_{}_4_false_false,0.0010,0.10000,0.000010,1,0.5,0.9,0.7219,28,0.7200,0.72095,"[2.1974, 0.6494, 0.6480, 0.6478, 0.6509, 0.650...","[0.6473, 0.6473, 0.6473, 0.6528, 0.6473, 0.647...","[0.6572, 0.6439, 0.6611, 0.6122, 0.6384, 0.640..."
1003,out_{}_4_false_false,0.0010,0.10000,0.000001,1,0.2,0.9,0.7212,29,0.7212,0.72120,"[2.0782, 0.6498, 0.6489, 0.6501, 0.6513, 0.650...","[0.6474, 0.6473, 0.6473, 0.6473, 0.6473, 0.649...","[0.6522, 0.6758, 0.6364, 0.6642, 0.6180, 0.617..."
1112,out_{}_4_false_false,0.0001,0.10000,0.000100,1,0.5,0.9,0.7210,27,0.7168,0.71890,"[2.5094, 0.6514, 0.6496, 0.6468, 0.6454, 0.646...","[0.6473, 0.6473, 0.6475, 0.6473, 0.6650, 0.656...","[0.7430, 0.6592, 0.6333, 0.6337, 0.6240, 0.611..."
986,out_{}_4_false_false,0.0010,0.10000,0.001000,1,0.5,0.9,0.7200,29,0.7200,0.72000,"[2.1806, 0.6475, 0.6496, 0.6490, 0.6512, 0.650...","[0.6475, 0.6473, 0.6473, 0.6473, 0.6473, 0.647...","[0.6239, 0.6472, 0.6557, 0.6494, 0.6286, 0.645..."
985,out_{}_4_false_false,0.0010,0.10000,0.001000,1,0.2,0.9,0.7199,29,0.7199,0.71990,"[2.0908, 0.6501, 0.6502, 0.6488, 0.6538, 0.651...","[0.6473, 0.6473, 0.6473, 0.6488, 0.6473, 0.677...","[0.6485, 0.6439, 0.6532, 0.6302, 0.6273, 0.603..."
1118,out_{}_4_false_false,0.0001,0.10000,0.000010,1,0.5,0.9,0.7196,27,0.7175,0.71855,"[2.5331, 0.6515, 0.6465, 0.6467, 0.6487, 0.645...","[0.6473, 0.6473, 0.6673, 0.6510, 0.6484, 0.681...","[0.6971, 0.6615, 0.6146, 0.6299, 0.6110, 0.590..."
992,out_{}_4_false_false,0.0010,0.10000,0.000100,1,0.5,0.9,0.7188,26,0.7186,0.71870,"[2.1796, 0.6493, 0.6501, 0.6497, 0.6529, 0.651...","[0.6473, 0.6473, 0.6473, 0.6473, 0.6473, 0.647...","[0.6428, 0.6305, 0.6491, 0.6495, 0.6339, 0.622..."
866,out_{}_4_false_false,0.0100,0.10000,0.001000,1,0.5,0.9,0.7184,29,0.7184,0.71840,"[1.2272, 0.6531, 0.6555, 0.6566, 0.6578, 0.662...","[0.6473, 0.6473, 0.6473, 0.6473, 0.6473, 0.647...","[0.6328, 0.6467, 0.6395, 0.6355, 0.6367, 0.643..."


wd
0.0010    17
0.0100    15
0.0001     8
Name: wd, dtype: int64

initial_lr
0.10000    23
0.01000     9
0.00001     3
0.00010     3
0.00100     2
Name: initial_lr, dtype: int64

eta_min
0.001000    13
0.000010    10
0.000001     9
0.000100     8
Name: eta_min, dtype: int64

num_hidden_layers
1    30
2    10
Name: num_hidden_layers, dtype: int64

dropout
0.5    24
0.2    10
0.1     6
Name: dropout, dtype: int64

momentum
0.9    40
Name: momentum, dtype: int64

out_{}_4_false_true


,dataset,wd,initial_lr,eta_min,num_hidden_layers,dropout,momentum,top_score,epoch_of_top_score,final_score,combined_score,train_loss,valid_acc,valid_loss
1292,out_{}_4_false_true,0.0010,0.00010,0.000001,1,0.5,0.9,0.6771,27,0.6771,0.67710,"[0.6779, 0.6453, 0.6305, 0.6195, 0.6115, 0.606...","[0.6566, 0.6606, 0.6647, 0.6633, 0.6672, 0.669...","[0.6293, 0.6231, 0.6198, 0.6170, 0.6141, 0.612..."
1199,out_{}_4_false_true,0.0100,0.00100,0.000001,2,0.5,0.9,0.6741,2,0.6300,0.65205,"[0.6566, 0.6177, 0.6072, 0.5974, 0.5856, 0.570...","[0.6670, 0.6711, 0.6741, 0.6700, 0.6663, 0.670...","[0.6142, 0.6089, 0.6081, 0.6039, 0.6058, 0.607..."
1187,out_{}_4_false_true,0.0100,0.00100,0.000100,2,0.5,0.9,0.6737,2,0.6047,0.63920,"[0.6554, 0.6189, 0.6062, 0.5972, 0.5862, 0.574...","[0.6690, 0.6714, 0.6737, 0.6736, 0.6679, 0.664...","[0.6148, 0.6135, 0.6093, 0.6085, 0.6088, 0.613..."
1244,out_{}_4_false_true,0.0100,0.10000,0.000001,1,0.5,0.9,0.6737,28,0.6717,0.67270,"[2.1177, 0.6379, 0.6386, 0.6386, 0.6388, 0.638...","[0.5063, 0.5121, 0.5083, 0.5096, 0.5123, 0.513...","[0.7428, 0.7163, 0.7371, 0.7274, 0.7276, 0.724..."
1232,out_{}_4_false_true,0.0100,0.10000,0.000100,1,0.5,0.9,0.6736,27,0.6727,0.67315,"[2.2832, 0.6376, 0.6388, 0.6386, 0.6385, 0.638...","[0.5086, 0.5107, 0.5099, 0.5125, 0.5111, 0.512...","[0.7376, 0.7256, 0.7177, 0.7312, 0.7158, 0.722..."
1247,out_{}_4_false_true,0.0100,0.10000,0.000001,2,0.5,0.9,0.6732,26,0.6609,0.66705,"[0.8523, 0.6471, 0.6481, 0.6481, 0.6488, 0.649...","[0.5193, 0.5144, 0.5044, 0.5115, 0.5096, 0.526...","[0.7114, 0.6959, 0.7042, 0.6979, 0.7150, 0.703..."
1289,out_{}_4_false_true,0.0010,0.00010,0.000010,2,0.5,0.9,0.6732,27,0.6727,0.67295,"[0.6947, 0.6593, 0.6446, 0.6384, 0.6325, 0.624...","[0.6611, 0.6617, 0.6642, 0.6670, 0.6676, 0.668...","[0.6238, 0.6207, 0.6197, 0.6171, 0.6154, 0.614..."
1236,out_{}_4_false_true,0.0100,0.10000,0.000010,1,0.1,0.9,0.6729,29,0.6729,0.67290,"[1.9676, 0.6379, 0.6389, 0.6395, 0.6388, 0.638...","[0.5137, 0.5128, 0.5128, 0.5161, 0.5162, 0.511...","[0.7240, 0.7185, 0.7246, 0.7198, 0.7246, 0.715..."
1379,out_{}_4_false_true,0.0001,0.00001,0.000100,2,0.5,0.9,0.6727,25,0.6723,0.67250,"[0.7393, 0.7080, 0.6922, 0.6874, 0.6833, 0.675...","[0.6105, 0.6370, 0.6499, 0.6525, 0.6572, 0.658...","[0.6613, 0.6444, 0.6366, 0.6325, 0.6292, 0.628..."
1230,out_{}_4_false_true,0.0100,0.10000,0.000100,1,0.1,0.9,0.6727,27,0.6694,0.67105,"[1.9592, 0.6381, 0.6387, 0.6386, 0.6391, 0.638...","[0.5158, 0.5133, 0.5136, 0.5169, 0.5157, 0.521...","[0.7120, 0.7191, 0.7179, 0.7108, 0.7190, 0.704..."


wd
0.0100    26
0.0001     7
0.0010     7
Name: wd, dtype: int64

initial_lr
0.10000    17
0.00100     9
0.00010     8
0.00001     6
Name: initial_lr, dtype: int64

eta_min
0.000010    11
0.000100    11
0.001000    10
0.000001     8
Name: eta_min, dtype: int64

num_hidden_layers
2    24
1    16
Name: num_hidden_layers, dtype: int64

dropout
0.5    28
0.1     7
0.2     5
Name: dropout, dtype: int64

momentum
0.9    40
Name: momentum, dtype: int64

out_{}_4_true_true


,dataset,wd,initial_lr,eta_min,num_hidden_layers,dropout,momentum,top_score,epoch_of_top_score,final_score,combined_score,train_loss,valid_acc,valid_loss
1604,out_{}_4_true_true,0.0100,0.10000,0.000001,1,0.5,0.9,0.6565,24,0.6420,0.64925,"[6.8048, 0.6216, 0.6204, 0.6201, 0.6189, 0.619...","[0.5567, 0.5656, 0.5713, 0.5671, 0.5646, 0.567...","[0.7320, 0.6975, 0.6915, 0.7011, 0.7035, 0.693..."
1592,out_{}_4_true_true,0.0100,0.10000,0.000100,1,0.5,0.9,0.6557,23,0.6393,0.64750,"[6.6264, 0.6211, 0.6199, 0.6197, 0.6196, 0.619...","[0.5545, 0.5710, 0.5679, 0.5675, 0.5684, 0.568...","[0.7419, 0.6937, 0.6994, 0.6943, 0.6895, 0.691..."
1586,out_{}_4_true_true,0.0100,0.10000,0.001000,1,0.5,0.9,0.6538,24,0.6385,0.64615,"[7.1800, 0.6207, 0.6201, 0.6202, 0.6196, 0.619...","[0.5542, 0.5654, 0.5659, 0.5640, 0.5657, 0.570...","[0.7309, 0.6932, 0.7005, 0.6959, 0.7060, 0.691..."
1769,out_{}_4_true_true,0.0001,0.00010,0.000010,2,0.5,0.9,0.6536,2,0.6393,0.64645,"[0.6807, 0.6380, 0.6132, 0.6007, 0.5910, 0.580...","[0.6385, 0.6478, 0.6536, 0.6524, 0.6528, 0.653...","[0.6575, 0.6439, 0.6374, 0.6359, 0.6371, 0.637..."
1598,out_{}_4_true_true,0.0100,0.10000,0.000010,1,0.5,0.9,0.6528,24,0.6418,0.64730,"[6.5908, 0.6206, 0.6197, 0.6206, 0.6203, 0.619...","[0.5439, 0.5700, 0.5675, 0.5700, 0.5746, 0.568...","[0.7541, 0.6924, 0.6915, 0.6852, 0.6865, 0.692..."
1584,out_{}_4_true_true,0.0100,0.10000,0.001000,1,0.1,0.9,0.6526,24,0.6331,0.64285,"[5.7388, 0.6205, 0.6195, 0.6190, 0.6187, 0.618...","[0.5752, 0.5708, 0.5671, 0.5715, 0.5686, 0.570...","[0.6920, 0.6873, 0.6983, 0.6896, 0.6964, 0.692..."
1591,out_{}_4_true_true,0.0100,0.10000,0.000100,1,0.2,0.9,0.6524,23,0.6289,0.64065,"[5.7084, 0.6197, 0.6196, 0.6189, 0.6190, 0.618...","[0.5816, 0.5706, 0.5654, 0.5671, 0.5702, 0.574...","[0.6918, 0.6892, 0.7015, 0.6952, 0.6947, 0.689..."
1597,out_{}_4_true_true,0.0100,0.10000,0.000010,1,0.2,0.9,0.6524,24,0.6378,0.64510,"[6.1331, 0.6196, 0.6192, 0.6188, 0.6189, 0.618...","[0.5547, 0.5692, 0.5694, 0.5686, 0.5737, 0.573...","[0.7042, 0.6851, 0.6909, 0.6979, 0.6889, 0.689..."
1603,out_{}_4_true_true,0.0100,0.10000,0.000001,1,0.2,0.9,0.6515,24,0.6351,0.64330,"[5.7273, 0.6193, 0.6197, 0.6191, 0.6188, 0.618...","[0.5704, 0.5681, 0.5704, 0.5704, 0.5731, 0.570...","[0.6997, 0.6902, 0.6902, 0.6969, 0.6849, 0.687..."
1774,out_{}_4_true_true,0.0001,0.00010,0.000001,2,0.2,0.9,0.6513,3,0.6248,0.63805,"[0.6513, 0.5958, 0.5647, 0.5398, 0.5149, 0.492...","[0.6385, 0.6445, 0.6461, 0.6513, 0.6503, 0.649...","[0.6561, 0.6479, 0.6467, 0.6473, 0.6513, 0.656..."


wd
0.0100    20
0.0001    12
0.0010     8
Name: wd, dtype: int64

initial_lr
0.10000    15
0.00001    12
0.00010    11
0.00100     2
Name: initial_lr, dtype: int64

eta_min
0.000001    12
0.001000    10
0.000010     9
0.000100     9
Name: eta_min, dtype: int64

num_hidden_layers
1    23
2    17
Name: num_hidden_layers, dtype: int64

dropout
0.5    18
0.2    13
0.1     9
Name: dropout, dtype: int64

momentum
0.9    40
Name: momentum, dtype: int64



In [104]:
best = df.loc[610]

In [1]:
best

NameError: name 'best' is not defined

In [111]:
pd.DataFrame(dict(best[['train_loss', 'valid_acc', 'valid_loss']]))

,train_loss,valid_acc,valid_loss
0,1.1758,0.4949,1.1151
1,1.0663,0.5146,1.0839
2,1.0217,0.5260,1.0696
3,0.9934,0.5333,1.0628
4,0.9764,0.5374,1.0587
5,0.9638,0.5387,1.0557
6,0.9529,0.5399,1.0531
7,0.9444,0.5412,1.0509
8,0.9382,0.5422,1.0490
9,0.9324,0.5433,1.0474
